<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/v2/05_deep_learning/04_transfer_learning/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transfer Learning

In the field of deep learning, **transfer learning** is defined as the conveyance of knowledge from one pretrained model to a new model. This simply means that transfer learning uses a pretrained model to train a new model. Typically the new model will have a more specific application than the pre-trained model.

*Note that this lab is largely based on [an excellent transfer learning lab](https://www.tensorflow.org/tutorials/images/transfer_learning) from TensorFlow.*

## Exploratory Data Analysis

The [`tensorflow_datasets`](https://www.tensorflow.org/datasets/api_docs/python/tfds) package has a [catalog of datasets](https://www.tensorflow.org/datasets/catalog/overview) that are easy to load into your TensorFlow environment for experimentation.

In this lab we'll work with the [`cats_vs_dogs`](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs) dataset. This dataset contains thousands of images of cats and dogs. Looking at the [documentation for the dataset](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs), we can see there are `23,262` examples in the 'train' split of data. There are no test and validation splits.

We could just load this one split directly and then split the data once we download it. Another option is to tell `tfds.load()` to split the data for us. To do that we must specify the splits.

There is a specific notation we can use that tells the function how much of the data we want in each split. For instance `'train[:80%]'` indicates that we want the first `80%` of the train split in one tranche. `'train[80%:90%]'` indicates that we want the next 10% of the data in another tranche, and so on. You can see this at work in our `split` example below.

In [0]:
import tensorflow_datasets as tfds

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

The metadata returned from our dataset contains useful information about the data. For instance, it includes the number of classes:

In [0]:
metadata.features['label'].num_classes

And the class names:

In [0]:
metadata.features['label'].names

It even comes with some handy functions for converting between class names and numbers:

In [0]:
print(metadata.features['label'].int2str(1))
print(metadata.features['label'].str2int('cat'))

Let's store the `int2str` into a more conveniently named function for later use.

In [0]:
get_class_name = metadata.features['label'].int2str
get_class_name(0), get_class_name(1)

Let's take a quick look at our dataset. First we'll peek at the shape of the data.

In [0]:
raw_train

`(None, None, 3)` lets us know that we have three channel images, but we aren't sure of the lengths and widths. They are likely different depending on the image. We also don't know how many images we have.

Let's do some deeper analysis.

It turns out that you can iterate over a `DatasetV1Adapter` with a standard `for` loop. The items returned at each iteration are the image and the label.

We'll create a helper function to analyze a split of our data.

In [0]:
import collections


def split_details(split):
  counts = collections.defaultdict(int)
  for image, label in split:
    counts[label.numpy()]+=1

  total = 0
  for cls, cnt in counts.items():
    print(f"Class {get_class_name(cls)}: {cnt}")
    total += cnt
  
  print(f"Total: {total}")

for s in (
    ("Train", raw_train),
    ("Validation", raw_validation),
    ("Test", raw_test)):
  print(s[0])
  split_details(s[1])
  print()

We'll train on `18,610` examples, validating on `2,326`, and performing our final testing on `2,326`. Our classes look pretty evenly spread across all of the splits. The classes also seem to have a similar number of total examples.

Let's now see what our images look like. We'll display one dog and one cat.

In [0]:
import matplotlib.pyplot as plt


for cls in (0, 1):
  for image, label in raw_train:
    if label == cls:
      plt.figure()
      plt.imshow(image)
      break

These are color images with noisy backgrounds. Also, the images aren't the same size, so we'll need to eventually resize them to feed our model.

Let's find the range of color values and image sizes.

In [0]:
import sys 

global_min = sys.maxsize 
global_max = -sys.maxsize-1
sizes = collections.defaultdict(int)

for split in (raw_train, raw_validation, raw_test):
  for image, _ in split:
    local_max = image.numpy().max()
    local_min = image.numpy().min()
    sizes[image.numpy().shape] += 1

    if local_max > global_max:
      global_max = local_max
    
    if local_min < global_min:
      global_min = local_min

print(f"Color values range from {global_min} to {global_max}")
resolutions = [x[0] for x in sorted(sizes.items(), key=lambda r: r[0])]
print(f"There are {len(resolutions)} resolutions ranging from ",
      f"{resolutions[0]} to {resolutions[-1]}")


It looks like we are dealing with color values from `0` through `255`, which is pretty standard.

We have a huge number of different resolutions. There are over `6,000` different image sizes in this dataset, some as small as `4x4x3`! It is difficult to imagine that an image that small would be meaningful. Let's see how many tiny images we are dealing with.


In [0]:
for resolution in sorted(sizes.items(), key=lambda r: r[0])[:10]:
  print(resolution[0], ': ', resolution[1])

There is only one truly tiny image. Let's take a look at it.

In [0]:
shown = False
for split in (raw_train, raw_validation, raw_test):
  if shown:
    break
  for image, _ in split:
    if image.numpy().shape == (4, 4, 3):
      plt.figure()
      plt.imshow(image)
      shown = True
      break

That's definitely bad data. Let's go ahead and sample some of the other small images.

In [0]:
for split in (raw_train, raw_validation, raw_test):
  for image, _ in split:
    if image.numpy().shape[0] < 50 and image.numpy().shape[0] > 4:
      plt.figure()
      plt.imshow(image)      

Though some are difficult to interpret, you can probably tell that each image contains either cats or dogs.

In order to not process the tiny image, we can write a filter function. We know the shape is `(4, 4, 3)`, so we can filter for that exact shape. To make the filter a little more generic, we'll instead filter out any image that is shorter or narrower than `6` pixels.

In [0]:
import tensorflow as tf

def filter_out_small(image, _):
  return tf.math.reduce_any(tf.shape(image)[0] > 5 and tf.shape(image)[1] > 5)

for s in (
    ("Train", raw_train.filter(filter_out_small)),
    ("Validation", raw_validation.filter(filter_out_small)),
    ("Test", raw_test.filter(filter_out_small))):
  print(s[0])
  split_details(s[1])
  print()

It looks like our problematic image was a cat in the holdout test set.

## The Pretrained Model

To build our cat/dog classifier, we'll use the learnings of a pre-trained model. Specifically [`MobileNetV2`](https://arxiv.org/abs/1801.04381). We'll use [`tf.keras.applications.MobileNetV2`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2) to load the data.

### Model-Specific Preprocessing

[Researching `MobileNetV2`](https://arxiv.org/pdf/1801.04381.pdf), you'll find that the neural network by default takes an input of image of size `(224, 224, 3)`. Though the model can be configured to take other inputs, all of our images are different sizes. So we might as well resize them to fit the default.

In [0]:
IMG_SIZE = 224 

def resize_images(image, label):
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train_resized = raw_train.map(resize_images)
validation_resized = raw_validation.map(resize_images)
test_resized = raw_test.map(resize_images)

We also need to normalize our data, but what should our input values be scaled to? Ideally our input data should look like the input data that the `MobileNetV2` was trained on. Unfortunately, this isn't published.

`MobileNetV2` [internally](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet_v2.py) uses [`relu6`](https://www.tensorflow.org/api_docs/python/tf/nn/relu6), which limits activation outputs to the range of `0` through `6`. This hints that we might want to normalize our values between `[0, 1]` or even `[0, 6]`.

It also performs **batch normalization** throughout the network. This is the process of dividing input values by the mean and subtracting the standard deviation of each batch of data processed. So "batch normalization" is really "batch standardization".

Standardizing our data by batch is possible. We could also calculate the mean and standard deviation of all of the data and standardize the entire dataset in one pass. Or we could *approximate* standardization and simply divide our input data by `127.5` (the midpoint of our `[0, 255]` range) and then subtract `1` (a guessed standard deviation).

In [0]:
def standardize_images(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train_standardized = train_resized.map(standardize_images)
validation_standardized = validation_resized.map(standardize_images)
test_standardized = test_resized.map(standardize_images)

Did it work? Let's check it out.

In [0]:
import sys 

global_min = sys.maxsize 
global_max = -sys.maxsize-1
sizes = collections.defaultdict(int)

for split in (train_standardized, validation_standardized, test_standardized):
  for image, _ in split:
    local_max = image.numpy().max()
    local_min = image.numpy().min()
    sizes[image.numpy().shape] += 1

    if local_max > global_max:
      global_max = local_max
    
    if local_min < global_min:
      global_min = local_min

print(f"Color values range from {global_min} to {global_max}")
resolutions = [x[0] for x in sorted(sizes.items(), key=lambda r: r[0])]
print(f"There are {len(resolutions)} resolutions ranging from ",
      f"{resolutions[0]} to {resolutions[-1]}")


Looks great! Now it is time to load our pretrained model.

### Loading MobileNetV2

Loading `MobileNetV2` is pretty straight-forward.

We need to pass in the input shape, which is `(224, 224, 3)` for each image.

We also include pre-trained weights based on [`ImageNet`](http://www.image-net.org/). This is where the transfer learning comes in. `ImageNet` has over a million images that map to a thousand labels. `MobileNetV2` has been trained on `ImageNet`. We'll use those learnings and then add a few more layers of our own model to build a cat/dog classifier.

The final argument is `include_top`. Typically when building a classification model, toward the end of the model, high-dimensional layers are flattened down into two-dimensional tensors. This is considered the top of the model since diagrams often show the final layers at the top. For transfer learning we'll leave this dimensionality reduction off.

If you do include the top of the model, the following extra layers will be shown:

```text    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1280)         0           out_relu[0][0]                   
__________________________________________________________________________________________________
predictions (Dense)             (None, 1000)         1281000     global_average_pooling2d_1[0][0] 
```

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

mnv2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                         weights='imagenet',
                                         include_top=False)

mnv2.summary()

It is often a good idea to "freeze" the trained model. This prevents its weights from being updated when we train our new model.

It is really only recommended to update the weights of the pretrained model when you are about to train on a large and similar dataset, as compared to the one that was originally trained on. This is not the case in our example. `ImageNet` has a thousand classes and over a million images. We have two classes and a few thousand images.

In [0]:
mnv2.trainable = False

### Batching

We will want to train our model in batches. In our case we'll use a batch size of `32`. You might want to experiment with other sizes.

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_standardized.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation_standardized.batch(BATCH_SIZE)
test_batches = test_standardized.filter(filter_out_small).batch(BATCH_SIZE)

You can see that we now have a well-defined input shape for each training batch.

In [0]:
image_batch, label_batch = next(iter(train_batches.take(1)))

image_batch.shape

If we apply our model to our first batch, you can see that we get a `(32, 7, 7, 1280)` block of features. These will be the input to our cat/dog model.

In [0]:
feature_batch = mnv2(image_batch)
print(feature_batch.shape)

### Extending the Model

Now we can perform the actual transfer learning. We'll build a new model that classifies images as containing dogs or cats. In order to do that, we can use a `Sequential` model with the pretrained model as the first layer.

Note that the output layer of our pretrained model is:

```text
out_relu (ReLU)                 (None, 7, 7, 1280)   0           Conv_1_bn[0][0]  
```

Since the activation function is `relu6`, we know that the data that we'll be receiving is in the range of `[0, 6]`. We apply a pooling layer to reduce our inputs. In our output layer, we distill the inputs down to a single number that indicates if the image is of a cat or dog. We chose the sigmoid function, which will cause the output to be in a range of `[0, 1]`. This represents the confidence in an image being a dog, since dog is encoded as `1`.



In [0]:
model = tf.keras.Sequential([
  mnv2,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

We now compile the model, training for accuracy with binary cross entropy used to calculate loss.

In [0]:
model.compile(
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy']
)

model.summary()

Training will take a few minutes. Be sure to use GPU, or it will take a really long time.

In [0]:
history = model.fit(
  train_batches,
  epochs=10,
  validation_data=validation_batches
)

We got a training accuracy of over `99%` and a validation accuracy close to `99%`! Let's graph the accuracy and loss per epoch.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

The graph makes it look like we might be overfitting, but if you look at the range on the y-axis, we actually aren't doing too badly. We should, however, perform a final test to see if we can generalize well.

In [0]:
model.evaluate(test_batches)

We got an accuracy of just over `99%`, which can give us some confidence that this model generalizes well.

### Making Predictions

We can use the model to make predictions by using the `predict()` function.

In [0]:
predictions = model.predict(test_batches)

predictions.min(), predictions.max()

Remember the predictions can range from `0.0` to `1.0`. We can round them and cast them to integers to get class mappings.

In [0]:
import numpy as np

predictions = np.round(predictions.flatten(), 0).astype(np.int)
predictions

And we can now print the predicted class alongside the original image.

In [0]:
print(get_class_name(predictions[0]))
_ = plt.imshow(next(iter(raw_test.take(1)))[0].numpy())

You can also make predictions by calling the model directly and passing it a single batch.

In [0]:
predictions = model(next(iter(test_batches)))
predictions = np.round(predictions, 0).astype(np.int).flatten()

print(get_class_name(predictions[0]))
_ = plt.imshow(next(iter(raw_test.take(1)))[0].numpy())

# Exercises

## Exercise 1: Food 101

In this exercise you'll build a classifier for the [Food 101](https://www.tensorflow.org/datasets/catalog/food101) dataset. The classifier will transfer learnings from [`DenseNet201`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/DenseNet201).

In order to complete the exercise, you will need to:
* Load the [Food 101](https://www.tensorflow.org/datasets/catalog/food101) dataset. *Be sure to pay attention to the splits!*
* Perform exploratory data analysis on the dataset.
* Ensure every class is represented in your train, test, and validation splits of the dataset.
* Normalize or standardize your data in the way that the model was trained. *You can find this information in the [paper introducing the model](https://arxiv.org/pdf/1608.06993.pdf).*
* Extend [`DenseNet201`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/DenseNet201) with a new model, and have it classify the `101` food types. *Note that [`one_hot`](https://www.tensorflow.org/api_docs/python/tf/one_hot) and [`Dataset.map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) can help you manipulate the targets to make the model train faster.*
* Graph training accuracy and loss.
* Calculate accuracy and loss for your holdout test set.*
* Make predictions and print out one predicted label and original image.

> **Don't sweat too much about your model's performance. We were only able to get about `75%` training accuracy (with obvious overfitting) in our naive model after `10` training epochs. This model is trying to classify `101` different things with messy images. Don't expect it to perform anywhere close to our binary model above.*

Use as many code and text cells as you need to complete this task. Explain your work.

### **Student Solution**

In [0]:
# Your Solution Goes Here

---

### Answer Key

We first load the data. Notice that we split the training set for train/validation, and then we use the validation split for our final holdout test.

In [0]:
import tensorflow_datasets as tfds

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'food101',
    split=['train[:90%]', 'train[90%:]', 'validation'],
    with_info=True,
    as_supervised=True,
)

Check out the feature names. Notice [many aren't in ImageNet](http://image-net.org/challenges/LSVRC/2012/browse-synsets).

In [0]:
metadata.features['label'].names

Create our class lookup.

In [0]:
get_class_name = metadata.features['label'].int2str

Ensure every label is represented in every dataset.

In [0]:
train_labels, validation_labels, test_labels = set(), set(), set()

for s in (
    (raw_train, train_labels),
    (raw_validation, validation_labels),
    (raw_test, test_labels)):
    for _, label in s[0]:
      s[1].add(label.numpy())

print(train_labels.symmetric_difference(validation_labels))
print(train_labels.symmetric_difference(test_labels))

Display a few images.

In [0]:
import matplotlib.pyplot as plt

for cls in range(10):
  for image, label in raw_train:
    if label == cls:
      plt.figure()
      plt.imshow(image)
      break

Check out the distribution of color and resolution. Resolution varies. Color looks fine.

In [0]:
import sys 

global_min = sys.maxsize 
global_max = -sys.maxsize-1
sizes = collections.defaultdict(int)

for split in (raw_train, raw_validation, raw_test):
  for image, _ in split:
    local_max = image.numpy().max()
    local_min = image.numpy().min()
    sizes[image.numpy().shape] += 1

    if local_max > global_max:
      global_max = local_max
    
    if local_min < global_min:
      global_min = local_min

print(f"Color values range from {global_min} to {global_max}")
resolutions = [x[0] for x in sorted(sizes.items(), key=lambda r: r[0])]
print(f"There are {len(resolutions)} resolutions ranging from ",
      f"{resolutions[0]} to {resolutions[-1]}")


Resize our images to the size that our pre-trained network expects.

In [0]:
IMG_SIZE = 224 

def resize_images(image, label):
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train_resized = raw_train.map(resize_images)
validation_resized = raw_validation.map(resize_images)
test_resized = raw_test.map(resize_images)

The original model performed whole-dataset standardization. You can do that, or approximate it like we did below.

In [0]:
def standardize_images(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train_standardized = train_resized.map(standardize_images)
validation_standardized = validation_resized.map(standardize_images)
test_standardized = test_resized.map(standardize_images)

Make sure that our standardization and resizing worked.

In [0]:
import sys 

global_min = sys.maxsize 
global_max = -sys.maxsize-1
sizes = collections.defaultdict(int)

for split in (train_standardized, validation_standardized, test_standardized):
  for image, _ in split:
    local_max = image.numpy().max()
    local_min = image.numpy().min()
    sizes[image.numpy().shape] += 1

    if local_max > global_max:
      global_max = local_max
    
    if local_min < global_min:
      global_min = local_min

print(f"Color values range from {global_min} to {global_max}")
resolutions = [x[0] for x in sorted(sizes.items(), key=lambda r: r[0])]
print(f"There are {len(resolutions)} resolutions ranging from ",
      f"{resolutions[0]} to {resolutions[-1]}")


Load `DenseNet201`.

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

densenet = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE,
                                             weights='imagenet',
                                             include_top=False)

densenet.summary()

Make it non-trainable. *It's okay if you made your pretrained model trainable, but we chose not to due to the diversity of datasets.*

In [0]:
densenet.trainable = False

Extend the model. We'll pool down to a dense layer with `101` outputs with a softmax activation applied. Each output is the confidence that the input matches a given class.

In [0]:
model = tf.keras.Sequential([
  densenet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(101, activation='softmax'),
])

model.summary()

Compile the model. We need to use `CategoricalCrossentropy` this time since we have a multi-class model.

In [0]:
model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy']
)

model.summary()

Set up batching.

In [0]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_standardized.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation_standardized.batch(BATCH_SIZE)
test_batches = test_standardized.batch(BATCH_SIZE)

Now we can train the model. We one-hot encode the targets to make it easier for the model to train. Training to an ordinal value between `0` and `101` is slow. Even the one-hot takes a while.

In [0]:
def make_one_hot(image, cls):
  return image, tf.one_hot(cls, 101)

history = model.fit(
  train_batches.map(make_one_hot),
  epochs=10,
  validation_data=validation_batches.map(make_one_hot)
)

Graph the accuracy and loss over each epoch.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

And now evaluate how well the model generalizes.

In [0]:
model.evaluate(test_batches.map(make_one_hot))

Print the first prediction.

In [0]:
predictions = model(next(iter(test_batches.map(make_one_hot))))

print(get_class_name(np.argmax(predictions[0].numpy())))
plt.figure()
_ = plt.imshow(next(iter(raw_test))[0].numpy())

---